In [1]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from scipy.stats import zscore
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# df["zone"] = df.apply(lambda row: "thermal" if (
#         row["climb_m(delta)"] > 0 or
#         row["climb_rate_m/s"] > 0)
#         else "standart", axis=1)

df = pd.read_csv(os.path.join("data", "flight_data.csv"), low_memory=False)

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341370 entries, 0 to 2341369
Data columns (total 26 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   filename                 object 
 1   datetime                 object 
 2   pilot                    object 
 3   latitude                 float64
 4   longitude                float64
 5   gps_altitude_m           int64  
 6   pressure_altitude_m      int64  
 7   previous_latitude        float64
 8   previous_longitude       float64
 9   distance_from_takeoff_m  float64
 10  distance_m               float64
 11  speed_km/s               float64
 12  climb_m                  float64
 13  climb_m(delta)           float64
 14  climb_rate_m/s           float64
 15  bearing                  int64  
 16  delta_bearing            float64
 17  glide_ratio              float64
 18  elapsed_time             float64
 19  zone                     object 
 20  temp                     float64
 21  pressure

e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\Lib\site-packages\numpy\_core\_methods.py:53: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,distance_m,speed_km/s,climb_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,...,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06,2.341370e+06
mean,3.843068e+01,3.196174e+01,2.176136e+03,2.624257e+03,3.843071e+01,3.196169e+01,1.209257e+05,1.557176e+01,inf,-1.122420e-02,...,1.453194e+02,1.121960e+01,5.469595e+00,1.352320e+04,3.079183e+01,1.007533e+03,2.686336e+01,8.987026e+00,3.198095e+00,2.131174e+02
std,1.047468e+00,2.632063e+00,2.018151e+03,7.251078e+02,1.047498e+00,2.632011e+00,8.752475e+04,1.632570e+02,NaN,5.855752e+00,...,9.240353e+01,1.675429e+01,7.916183e+00,8.041481e+03,3.559163e+00,2.608612e+00,7.285804e+00,3.105875e+00,1.352500e+00,1.200338e+02
min,3.650395e+01,2.463248e+01,-7.347000e+03,0.000000e+00,3.650395e+01,2.463248e+01,0.000000e+00,0.000000e+00,-5.680640e+01,-6.668000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.838000e+01,9.990000e+02,6.000000e+00,-5.280000e+00,0.000000e+00,0.000000e+00
25%,3.746615e+01,3.003393e+01,1.130000e+03,2.121000e+03,3.746613e+01,3.003387e+01,4.524571e+04,1.039848e+01,3.951184e+01,-1.000000e+00,...,6.300000e+01,2.000000e+00,0.000000e+00,6.647000e+03,2.841000e+01,1.006000e+03,2.200000e+01,7.000000e+00,2.200000e+00,1.090000e+02
50%,3.838473e+01,3.200072e+01,2.276000e+03,2.624000e+03,3.838478e+01,3.200073e+01,1.081036e+05,1.415801e+01,5.130156e+01,0.000000e+00,...,1.500000e+02,6.000000e+00,3.638644e+00,1.332500e+04,3.051000e+01,1.007000e+03,2.700000e+01,9.280000e+00,3.230000e+00,2.410000e+02
75%,3.918790e+01,3.267807e+01,2.859000e+03,3.108000e+03,3.918800e+01,3.267802e+01,1.920118e+05,1.718033e+01,6.193115e+01,1.000000e+00,...,2.060000e+02,1.500000e+01,8.803139e+00,1.998500e+04,3.313000e+01,1.009000e+03,3.200000e+01,1.111000e+01,4.060000e+00,3.170000e+02
max,4.272443e+01,4.104910e+01,1.619900e+04,9.999000e+03,4.272443e+01,4.104910e+01,3.681782e+05,2.484829e+05,inf,1.428000e+03,...,3.590000e+02,1.800000e+02,5.176727e+03,3.667000e+04,4.111000e+01,1.018000e+03,5.800000e+01,2.225000e+01,8.700000e+00,3.600000e+02


In [4]:
def zscore_calc(df):
    column_names = df.columns.to_list()
    for column_name in column_names:
        df["zscore_" + column_name] = zscore(df[column_name])
    column_names = [name for name in df.columns if name.startswith("zscore")]
    def findzscore(row):
        for column_name in column_names:
            if abs(row[column_name]) > 3:
                return True
        return False
    # df["deleted"] = df.apply(findzscore, axis=1)
    return df.apply(findzscore, axis=1)
    

# "gps_altitude_m", "pressure_altitude_m", "distance_from_takeoff_m", "distance_m", "speed_km/s", "climb_m", "climb_m(delta)", "climb_rate_m/s", "glide_ratio"
df["zscore"] = zscore_calc(df[["gps_altitude_m", "pressure_altitude_m", "distance_from_takeoff_m", "distance_m", "speed_km/s", "climb_m", "climb_m(delta)", "climb_rate_m/s", "glide_ratio"]].copy())

# df["gps_altitude_m"] = np.where(
#     (df["gps_altitude_m"] < 0) | (abs(zscore(df["gps_altitude_m"])) > 3),
#     df["pressure_altitude_m"],
#     df["gps_altitude_m"]
# )
# df["pressure_altitude_m"] = np.where(
#     (df["pressure_altitude_m"] < 0) | (abs(zscore(df["pressure_altitude_m"])) > 3),
#     df["gps_altitude_m"],
#     df["pressure_altitude_m"]
# )

# df.drop(["gpszscore", "presurezscore"], axis=1, inplace=True)
df = df[~(((df["gps_altitude_m"] == 0) & (df["pressure_altitude_m"] == 0)) | (df["zscore"] == True))]
df.drop("zscore", axis=1, inplace=True)
df.head()

e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\Lib\site-packages\scipy\stats\_stats_py.py:1127: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean
e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\Lib\site-packages\numpy\_core\_methods.py:136: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,...,delta_bearing,glide_ratio,elapsed_time,zone,temp,pressure,humidity,dew_point,wind_speed,wind_deg
0,file_88076_2021-07-31-XFH-000-01.IGC,2021-07-31 08:36:09,egemen bilge,40.031117,32.328200,1137,1138,40.031117,32.328233,3.394062,...,0.0,0.000000,18.0,standart,29.66,1010.0,29.0,9.75,1.26,22.0
1,file_88076_2021-07-31-XFH-000-01.IGC,2021-07-31 08:36:10,egemen bilge,40.031133,32.328167,1134,1137,40.031117,32.328200,5.690337,...,33.0,1.131354,19.0,standart,29.66,1010.0,29.0,9.75,1.26,22.0
2,file_88076_2021-07-31-XFH-000-01.IGC,2021-07-31 08:36:11,egemen bilge,40.031150,32.328100,1135,1137,40.031133,32.328167,11.530152,...,15.0,5.983695,20.0,thermal,29.66,1010.0,29.0,9.75,1.26,22.0
3,file_88076_2021-07-31-XFH-000-01.IGC,2021-07-31 08:36:12,egemen bilge,40.031183,32.328033,1139,1137,40.031150,32.328100,17.951081,...,15.0,1.697031,21.0,thermal,29.66,1010.0,29.0,9.75,1.26,22.0
4,file_88076_2021-07-31-XFH-000-01.IGC,2021-07-31 08:36:13,egemen bilge,40.031183,32.327967,1139,1138,40.031183,32.328033,23.428631,...,33.0,0.000000,22.0,thermal,29.66,1010.0,29.0,9.75,1.26,22.0


In [5]:
df.describe()

e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
e:\Wolf\Belgeler\Projelerim\Paragliding_Flight_Analyses\.venv\Lib\site-packages\numpy\_core\_methods.py:53: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,distance_m,speed_km/s,climb_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
count,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,...,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06,2.240978e+06
mean,3.841143e+01,3.189752e+01,1.888793e+03,2.636300e+03,3.841145e+01,3.189748e+01,1.210320e+05,1.465871e+01,inf,9.354844e-03,...,1.449371e+02,1.099354e+01,5.302782e+00,1.358014e+04,3.087516e+01,1.007417e+03,2.676903e+01,9.000035e+00,3.143329e+00,2.109754e+02
std,1.051689e+00,2.638186e+00,1.226463e+03,7.261917e+02,1.051717e+00,2.638143e+00,8.795054e+04,1.126084e+01,NaN,1.864392e+00,...,9.338853e+01,1.583149e+01,6.066277e+00,8.077436e+03,3.524941e+00,2.490798e+00,7.315545e+00,3.096391e+00,1.338485e+00,1.195082e+02
min,3.650395e+01,2.463248e+01,0.000000e+00,4.490000e+02,3.650395e+01,2.463248e+01,0.000000e+00,0.000000e+00,-5.680640e+01,-1.700000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.838000e+01,9.990000e+02,6.000000e+00,-5.280000e+00,0.000000e+00,0.000000e+00
25%,3.744745e+01,2.998193e+01,9.050000e+02,2.128000e+03,3.744742e+01,2.998189e+01,4.492051e+04,1.036509e+01,3.950947e+01,-1.000000e+00,...,6.000000e+01,2.000000e+00,0.000000e+00,6.667000e+03,2.845000e+01,1.006000e+03,2.200000e+01,7.050000e+00,2.130000e+00,9.500000e+01
50%,3.835253e+01,3.168385e+01,2.231000e+03,2.637000e+03,3.835255e+01,3.168388e+01,1.077363e+05,1.404850e+01,5.120323e+01,0.000000e+00,...,1.490000e+02,6.000000e+00,3.698814e+00,1.338700e+04,3.060000e+01,1.007000e+03,2.700000e+01,9.280000e+00,3.150000e+00,2.390000e+02
75%,3.915717e+01,3.261890e+01,2.809000e+03,3.126000e+03,3.915725e+01,3.261887e+01,1.926643e+05,1.703664e+01,6.190683e+01,1.000000e+00,...,2.070000e+02,1.500000e+01,8.836344e+00,2.008900e+04,3.320000e+01,1.009000e+03,3.200000e+01,1.111000e+01,3.990000e+00,3.140000e+02
max,4.272443e+01,4.104910e+01,4.600000e+03,4.799000e+03,4.272443e+01,4.104910e+01,3.681782e+05,3.747842e+02,inf,1.700000e+01,...,3.590000e+02,1.800000e+02,2.921359e+01,3.667000e+04,4.111000e+01,1.018000e+03,5.800000e+01,2.225000e+01,8.700000e+00,3.600000e+02


In [ ]:
# Create a figure with larger size
plt.figure(figsize=(15, 8))

# Create boxplot for key numerical columns
cols_to_plot = ['gps_altitude_m', 'pressure_altitude_m', 'distance_from_takeoff_m', 
                'distance_m', 'speed_km/s', 'climb_m', 'climb_m(delta)', 
                'climb_rate_m/s', 'glide_ratio']

# Create boxplot using seaborn
sns.boxplot(data=df[cols_to_plot])

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add title
plt.title('Distribution of Flight Parameters')

# Adjust layout to prevent label cutoff
plt.tight_layout()

In [12]:
df = df[df["filename"] == "file_17415_2013-08-04_Basat_Ayas_251km.igc"]
df.head()

,filename,datetime,pilot,latitude,longitude,gps_altitude_m,pressure_altitude_m,previous_latitude,previous_longitude,distance_from_takeoff_m,...,bearing,delta_bearing,glide_ratio,elapsed_time,temp,pressure,humidity,dew_point,wind_speed,wind_deg
1833032,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:06,Basat,40.032200,32.326233,1048,779,40.032250,32.326267,413.117198,...,207,13.0,6.238333,15.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833033,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:07,Basat,40.032233,32.326167,1048,792,40.032200,32.326233,419.833207,...,303,96.0,0.000000,16.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833034,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:08,Basat,40.032300,32.326133,1047,799,40.032233,32.326167,425.552413,...,339,36.0,7.930290,17.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833035,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:09,Basat,40.032350,32.326117,1048,806,40.032300,32.326133,429.248951,...,345,6.0,5.731121,18.0,26.71,1013.0,29.0,7.22,2.79,354.0
1833036,file_17415_2013-08-04_Basat_Ayas_251km.igc,2013-08-04 09:34:10,Basat,40.032400,32.326083,1047,815,40.032350,32.326117,434.261542,...,332,13.0,6.238330,19.0,26.71,1013.0,29.0,7.22,2.79,354.0
